## Import libraries and load files

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import SCORERS

from sklearn.model_selection import KFold
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from warnings import warn

import numpy as np
import pandas as pd

In [2]:
#Read the data using the Unnamed (probably id) as index
url = 'https://s3.amazonaws.com/drivendata/data/4/public/81e8f2de-9915-4934-b9ae-9705685c9d50.csv'
#url = '../src/data/raw/training.csv'
training = pd.read_csv(url, index_col='Unnamed: 0')

labels = ['Function', 'Object_Type', 'Operating_Status', 'Position_Type', 'Pre_K', 'Reporting', 
          'Sharing', 'Student_Type', 'Use']

numeric = ['FTE', 'Total']

categoric = [ 'Facility_or_Department', 'Function_Description', 
            'Fund_Description', 'Job_Title_Description', 'Location_Description', 
            'Object_Description', 'Position_Extra', 'Program_Description', 'SubFund_Description', 
            'Sub_Object_Description', 
            'Text_1', 'Text_2', 'Text_3', 'Text_4']

### FunctionTransformers

In [3]:
# Define combine_text_columns()
def combine_text_columns(data_frame):
    """ converts all text in each row of data_frame to single vector """
    
    # Drop non-text columns that are in the df
    text_data = data_frame[categoric]
    
    # Replace nans with blanks
    text_data.fillna("", inplace=True)
    
    for category in categoric:
      training.loc[:,category] = training[category].str.lower()
    
    
    # Join all text items in a row that have a space in between
    return text_data.apply(lambda x: " ".join(x), axis=1)

In [4]:
groupped_FTE = training[['FTE', 'Object_Type']].groupby(by='Object_Type')
groupped_total = training[['Total', 'Object_Type']].groupby(by='Object_Type')
# Define combine_numeric_columns()
def combine_numeric_columns(data_frame, groupped_FTE=groupped_FTE, groupped_total=groupped_total):
    """ process all the numeric data """
    
    # Drop non-numeric columns that are in the df
    data = data_frame[numeric]
    
    #Remove inconsistent data
    data.loc[(data[numeric[0]] < 0) | (data[numeric[0]] > 1), numeric[0]] = np.nan
    data.loc[(data[numeric[1]] > 1) | (data[numeric[1]] < 0), numeric[1]] = np.nan
    
    #Impute the missing data with the median from each class
    for group in groupped_FTE.median().index:
      indexes_FTE = groupped_FTE.get_group(group).index.values
      indexes_total = groupped_total.get_group(group).index.values
      data.loc[ data.FTE.isnull() & np.isin(data.index.values,indexes_FTE), 'FTE'] = groupped_FTE.median().loc[group, "FTE"]
      data.loc[ data.Total.isnull() & np.isin(data.index.values,indexes_total), 'Total'] = groupped_total.median().loc[group,"Total"]
      
    return data

In [5]:
# Preprocess the text data: get_text_data
get_text_data = FunctionTransformer(combine_text_columns, validate=False)

# Preprocess the numeric data: get_numeric_data
get_numeric_data = FunctionTransformer(combine_numeric_columns, validate=False)



In [6]:
# Recover the targets and split the data
y = pd.get_dummies(training['Object_Type'])
X = training.drop(columns=labels)

### Pipeline

Apply the transformations on numeric and categorica data. Neither dimension reduction or standard scaler are used.

In [7]:
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imp', SimpleImputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer',HashingVectorizer(token_pattern="[A-Za-z0-9]+(?=\\s+)", 
                                                    norm=None, 
                                                    binary=False,
                                                    ngram_range=(1,2)) 
                    )
                ]))
             ]
        )),
        ('reduce_dim', TruncatedSVD(n_components = 50)),
        ('scaler', StandardScaler()),
        ('clf', OneVsRestClassifier(KNeighborsClassifier(n_neighbors=2)))
        
    ])

Applying the steps, we got a sparse matrix with 1048578 features.

## Training
The model is trained and tested using k-fold with k = 10. The GridSearchCV.

In [8]:
# create a dictionary of all values we want to test for n_neighbors
# Test k = 3, k = 5, k = 7
param_grid = {'clf__estimator__n_neighbors': np.arange(3, 9, 2)}

# Metrics: acc, 
metricas = [ 'accuracy',
 'average_precision',
# 'balanced_accuracy',
# 'precision',
# 'precision_samples',
# 'recall',
 'roc_auc',
           ]

#use gridsearch to test all values for n_neighbors
knn_gscv = GridSearchCV(pl, param_grid=param_grid,
                    scoring=metricas,
                    verbose=1,
                    refit="average_precision",
                    return_train_score=False, cv=10)

#fit model to data
knn_gscv.fit(X, y)

Fitting 10 folds for each of 3 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#in

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('union', FeatureUnion(n_jobs=None,
       transformer_list=[('numeric_features', Pipeline(memory=None,
     steps=[('selector', FunctionTransformer(accept_sparse=False, check_inverse=True,
          func=<function combine_numeric_columns at 0x7fe6a7ad7158>,
          inv_kw_args=None, invers...arams=None, n_jobs=None, n_neighbors=2, p=2,
           weights='uniform'),
          n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'clf__estimator__n_neighbors': array([3, 5, 7])},
       pre_dispatch='2*n_jobs', refit='average_precision',
       return_train_score=False,
       scoring=['accuracy', 'average_precision', 'roc_auc'], verbose=1)

In [9]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

In [10]:
results = pd.DataFrame(knn_gscv.cv_results_)

In [11]:
results.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__estimator__n_neighbors,params,split0_test_accuracy,split1_test_accuracy,split2_test_accuracy,split3_test_accuracy,...,split3_test_roc_auc,split4_test_roc_auc,split5_test_roc_auc,split6_test_roc_auc,split7_test_roc_auc,split8_test_roc_auc,split9_test_roc_auc,mean_test_roc_auc,std_test_roc_auc,rank_test_roc_auc
0,115.485431,7.331943,524.280141,37.067851,3,{'clf__estimator__n_neighbors': 3},0.989083,0.989208,0.988358,0.988958,...,0.990401,0.989061,0.988554,0.988228,0.989051,0.989027,0.989813,0.989199,0.000952,3
1,107.080926,7.502075,638.979905,20.941693,5,{'clf__estimator__n_neighbors': 5},0.986385,0.987209,0.986235,0.986659,...,0.992855,0.991815,0.991163,0.991168,0.991887,0.991985,0.992498,0.992060,0.000873,2
2,106.348304,4.958070,4649.533020,11527.262978,7,{'clf__estimator__n_neighbors': 7},0.984161,0.984836,0.983911,0.984786,...,0.994612,0.993098,0.993193,0.992537,0.993658,0.993145,0.993767,0.993500,0.000740,1


In [12]:
results.to_csv('../reports/knn_results_scaler.csv')